# AGF-345
Polar Magnetospheric Substorms

## Auroral Acceleration - Data Fetcher

### Authors

 - Martin Claude Joseph Baudry
 - Jeanne Longlune
 - Michał Jan Odorczuk
 - Inés Eleonor Santandreu Ros
 - Ana Filipa Sousa Barros

### Description

This notebook automates downloading all the necessary data for the project analysis.

### Necessary imports

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

### Tsyganenko modelled magnetic field lines

Automatically fetches the data from [here](https://ccmc.gsfc.nasa.gov/requests/instant/tsyganenko.php) and saves it in the data/Tsy/(DAY)_(HOUR) location, where DAY is the day of the year (assumed to be 2015 in the model) and HOUR is the hour of the day.

#### Invariant metadata

In [2]:
# URL to which the form is submitted
url = "https://ccmc.gsfc.nasa.gov/requests/instant/tsyganenko_results.php"

# Headers (based on your provided manual POST request)
headers = {
    'Host': 'ccmc.gsfc.nasa.gov',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://ccmc.gsfc.nasa.gov',
    'Connection': 'keep-alive',
    'Referer': 'https://ccmc.gsfc.nasa.gov/requests/instant/tsyganenko.php',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Priority': 'u=0',
}

locations = {
    'LYR': (78.222, 15.648),
    'BJR': (74.443, 19.016),
    'TRM': (69.650, 18.955)
}

#### External data

In [3]:
omni_fmt = pd.read_fwf("data/omni.fmt", skiprows=4, index_col = 0, names = ["index", "name", "format"])
omni_fmt['name'] = omni_fmt['name'].str.split(',').str[0]
omni_data = pd.read_fwf("data/omni.lst", names = omni_fmt["name"])



def get_omni_data(day, hour, time_shift = True):
    if time_shift:
        hour -= 1
    if hour < 0:
        day -= 1
        hour = 23
    data = omni_data[(omni_data['DOY'] == day)&(omni_data['Hour'] == hour)].iloc[0]
    return {"By": data["BY"], "Bz": data["BZ"], "v": data["SW Plasma Speed"], "p": data["Flow pressure"], "Dst": data["Dst-index"]}

#### Variables

In [ ]:
location = 'LYR'
lat, lon = locations[location]

def get_request_data(day, hour, location, model_version, time_shift = True):
    lat, lon = locations[location]
    omni_data = get_omni_data(day, hour, time_shift)
    return {
        'ts_version': model_version,
        'Year': '2015',
        'Day': day,
        'Hour': hour,
        'Minute': '00',
        'Second': '00',
        'SW dynamic pressure': omni_data['p'],
        'SW velocity': omni_data['v'],
        'IMF By': omni_data['By'],
        'IMF Bz': omni_data['Bz'],
        'Dst': omni_data['Dst'],
        'DIR': '1',
        'Geographic Geocentric Latitude': lat,
        'Longitude': lon,
        'Xgsm': '1.',
        'Ygsm': '1.',
        'Zgsm': '1.',
    }

In [ ]:
def fetch_datafile(day, hour, location, model_version='01', time_shift = True):
    # Send the POST request
    data = get_request_data(day, hour, location, model_version, time_shift)
    response = requests.post(url, headers=headers, data=data)

    # Parse the HTML response to find the download link
    soup = BeautifulSoup(response.content, 'html.parser')
    link = None
    for a in soup.find_all('a', href=True):
        if "MF_LINE_GSM" in a['href']:
            link = a['href']
            break

    if link:
        # Ensure the link is absolute
        download_url = f"https://ccmc.gsfc.nasa.gov{link}"

        # Download the file
        file_response = requests.get(download_url)
        if file_response.status_code == 200:
            # Save the file
            if not os.path.isdir("data"):
                os.mkdir("data")
            if not os.path.isdir("data/Tsy"):
                os.mkdir("data/Tsy")
            parent = f"data/Tsy/{day}_{hour:02d}"
            if not os.path.isdir(parent):
                os.mkdir(parent)
            filename = parent + f"/B_{day}_2015_T{model_version}_{location}_{hour:02d}.txt"
            with open(filename, 'wb') as file:
                file.write(file_response.content)
            print(f"File downloaded successfully and saved as {filename}")
        else:
            print(f"Failed to download the file, status code: {file_response.status_code}")
    else:
        print("Download link not found in the response.")

In [ ]:
for hour in [0, 17]:
    for location in locations.keys():
        fetch_datafile(311, hour, location, model_version='01', time_shift=True)

File downloaded successfully and saved as data/Tsy/311_00/B_311_2015_T01_LYR_00.txt
File downloaded successfully and saved as data/Tsy/311_00/B_311_2015_T01_BJR_00.txt
File downloaded successfully and saved as data/Tsy/311_00/B_311_2015_T01_TRM_00.txt
File downloaded successfully and saved as data/Tsy/311_17/B_311_2015_T01_LYR_17.txt
File downloaded successfully and saved as data/Tsy/311_17/B_311_2015_T01_BJR_17.txt
File downloaded successfully and saved as data/Tsy/311_17/B_311_2015_T01_TRM_17.txt
